## Thanksgiving Dinner Dataset
In this project, we will be working on Thanksgiving Dinner dataset published by FiveThirtyEight.

We'd like to know what people ate for Thanksgivng.



In [8]:
import pandas as pd
data=pd.read_csv('thanksgiving.csv',encoding='Latin-1')
data.head(3)

,RespondentID,Do you celebrate Thanksgiving?,What is typically the main dish at your Thanksgiving dinner?,What is typically the main dish at your Thanksgiving dinner? - Other (please specify),How is the main dish typically cooked?,How is the main dish typically cooked? - Other (please specify),What kind of stuffing/dressing do you typically have?,What kind of stuffing/dressing do you typically have? - Other (please specify),What type of cranberry saucedo you typically have?,What type of cranberry saucedo you typically have? - Other (please specify),...,Have you ever tried to meet up with hometown friends on Thanksgiving night?,"Have you ever attended a ""Friendsgiving?""",Will you shop any Black Friday sales on Thanksgiving Day?,Do you work in retail?,Will you employer make you work on Black Friday?,How would you describe where you live?,Age,What is your gender?,How much total combined money did all members of your HOUSEHOLD earn last year?,US Region
0,4337954960,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,None,NaN,...,Yes,No,No,No,NaN,Suburban,18 - 29,Male,"$75,000 to $99,999",Middle Atlantic
1,4337951949,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Other (please specify),Homemade cranberry gelatin ring,...,No,No,Yes,No,NaN,Rural,18 - 29,Female,"$50,000 to $74,999",East South Central
2,4337935621,Yes,Turkey,NaN,Roasted,NaN,Rice-based,NaN,Homemade,NaN,...,Yes,Yes,Yes,No,NaN,Suburban,18 - 29,Male,"$0 to $9,999",Mountain


In [9]:
print('The shape of dataset is: ',data.shape)
data.columns[0:5] #to get a sense of how the dataset look like.

The shape of dataset is:  (1058, 65)


Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?'],
      dtype='object')

In [10]:
data.loc[:,'Do you celebrate Thanksgiving?'].value_counts()

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

## let's filtering out those who did not celebrate

In [11]:
# we will remove data records of people who did not celebrate Thanksgiving
data.iloc[:,1].value_counts() #count how many times Yes and No happens
##We can also select this row using loc indexing:
#data.loc[:,'Do you celebrate Thanksgiving?'].value_counts()

celebrate=data.loc[:,'Do you celebrate Thanksgiving?']
yes_celebrate=celebrate=='Yes'
data=data[yes_celebrate] #only keeping yes values

In [12]:
data.loc[:,'What is typically the main dish at your Thanksgiving dinner?'].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

## How many put gravy on their tofurkey!

In [15]:
#Displaying number of poeple who had tofurkey and if they used gravy
is_turkey=data.loc[:,'What is typically the main dish at your Thanksgiving dinner?']=='Tofurkey'
is_gravey_column=data.loc[:,'Do you typically have gravy?'][is_turkey]
is_gravey_column.value_counts()

Yes    12
No      8
Name: Do you typically have gravy?, dtype: int64

## How many did not have any pies?

In [14]:
apple_isnull=pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple'])
pumkin_isnull=pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin'])
pecan_isnull=pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan'])
ate_pies=(apple_isnull)&(pumkin_isnull)&(pecan_isnull)
ate_pies.value_counts()

False    876
True     104
dtype: int64

## Converting age ranges to int values
### First we define a function that splits the range and take the first one, and replaces the 60+ with 60

In [21]:
def age_to_num(input):
    if pd.isnull(input):
        return None
    splited_list=input.split(' ')
    age=splited_list[0]
    age=age.replace('+','')
    return int(age)

In [27]:
data['int_age']=data["Age"].apply(age_to_num)
data["int_age"].describe()

count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%       30.000000
50%       45.000000
75%       60.000000
max       60.000000
Name: int_age, dtype: float64

In [38]:
def extract_income(income_str):
    if pd.isnull(income_str):
        return None
    income_str = income_str.split(" ")[0]
    if income_str == "Prefer":
        return None
    income_str = income_str.replace(",", "")
    income_str = income_str.replace("$", "")
    return int(income_str)

data['int_income']=data['How much total combined money did all members of your HOUSEHOLD earn last year?'].apply(extract_income)
data['int_income'].describe()

count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

## Figuring out transportation distances relative to income level

In [50]:
under_150k=data['int_income']<150000
travel=data['How far will you travel for Thanksgiving?'][under_150k]
a=travel.value_counts()
print('Travel Distance for under 150k income: ',a)

above_150k=data['int_income']>150000
travel_above=data['How far will you travel for Thanksgiving?'][above_150k]
b=travel_above.value_counts()
print('Travel Distance for above 150k income: ',b)

Travel Distance for under 150k income:  Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64
Travel Distance for above 150k income:  Thanksgiving is happening at my home--I won't travel at all                         49
Thanksgiving is local--it will take place in the town I live in                     25
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    16
Thanksgiving is out of town and far away--I have to drive several hours or fly      12
Name: How far will you travel for Thanksgiving?, dtype: int64


## Findings:
people with a lower income traveled more often, probably becuase higher income poeple host dinners!

## Let's figure out the age range for friendsgiving!

In [52]:
data.pivot_table(index="Have you ever tried to meet up with hometown friends on Thanksgiving night?",columns='Have you ever attended a "Friendsgiving?"',values="int_age")

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


## Findings
Younger people seem to stay the thanksgiving together more often.